## Introduction

Pymatgen is a very useful Python package for Materials research. This notebook demonstrates the core functionality of pymatgen, including the core objects representing Elements, Species, Lattices, and Structures. Adapted from the [Materials Virtual Lab](http://matgenb.materialsvirtuallab.org/) collection of Jupyter notebooks.

Written using:
- pymatgen==2019.3.13

By convention, we import pymatgen as mg.

In [1]:
import pymatgen as mg

## Basic Element, Specie and Composition objects

Pymatgen contains a set of core classes to represent an Element, Specie and Composition. These objects contains useful properties such as atomic mass, ionic radii, etc. These core classes are loaded by default with pymatgen. 

### Element
An [Element](http://pymatgen.org/pymatgen.core.periodic_table.html?highlight=element#pymatgen.core.periodic_table.Element) can be created as follows:

In [2]:
si = mg.Element("Si")
print("Atomic mass of Si is {}".format(si.atomic_mass))
print("Si has a melting point of {}".format(si.melting_point))
print("Ionic radii for Si: {}".format(si.ionic_radii))

Atomic mass of Si is 28.0855 amu
Si has a melting point of 1687.0 K
Ionic radii for Si: {4: 0.54}


You can see that units are printed for atomic masses and ionic radii. Pymatgen comes with a complete system of managing units in pymatgen.core.unit. A Unit is a subclass of float that attaches units and handles conversions. For example,

In [3]:
print("Atomic mass of Si in kg: {}".format(si.atomic_mass.to("kg")))

Atomic mass of Si in kg: 4.6637069207919995e-26 kg


Please refer to the [Units example](https://github.com/materialsvirtuallab/matgenb/blob/master/notebooks/2013-01-01-Units.ipynb) for more information on units.

### Specie
[Species](http://pymatgen.org/pymatgen.core.periodic_table.html?highlight=specie#pymatgen.core.periodic_table.Specie) are like Elements, except they have an explicit oxidation state. They can be used wherever Element is used for the most part.

In [4]:
fe2 = mg.Specie("Fe", 2)
print(fe2.atomic_mass)
print(fe2.ionic_radius)

55.845 amu
0.92 ang


An example of how the addition of the oxidation state can be useful is the `get_crystal_field_spin` method, which allows you to find the expected number of unpaired spins based on [Crystal Field Theory](https://en.wikipedia.org/wiki/Crystal_field_theory):

In [5]:
fe2.get_crystal_field_spin()

4

---
#### Exercise 1

Set up a Mn `Specie` with oxidation state +3 in a console tied to this notebook. Try using the `get_crystal_field_spin` and check the result. Using the `Shift+Tab` functionality of JupyterLab, figure out what the standard coordination and field splitting (`spin_config`) are, and try the other field splitting setting.

---

### Composition

A [Composition](http://pymatgen.org/pymatgen.core.composition.html?highlight=composition#pymatgen.core.composition.Composition) is essentially an **immutable** mapping of Elements/Species with amounts, and useful properties like molecular weight, get_atomic_fraction, etc. Note that you can conveniently either use an Element/Specie object or a string as keys (this is a feature).

In [6]:
comp = mg.Composition("Fe2O3")
print("Weight of Fe2O3 is {}".format(comp.weight))
print("Amount of Fe in Fe2O3 is {}".format(comp["Fe"]))
print("Atomic fraction of Fe is {}".format(comp.get_atomic_fraction("Fe")))
print("Weight fraction of Fe is {}".format(comp.get_wt_fraction("Fe")))

Weight of Fe2O3 is 159.6882 amu
Amount of Fe in Fe2O3 is 2.0
Atomic fraction of Fe is 0.4
Weight fraction of Fe is 0.699425505453753 


## Lattice & Structure objects

A [Lattice](http://pymatgen.org/pymatgen.core.lattice.html?highlight=lattice#pymatgen.core.lattice.Lattice) represents a Bravais lattice. Convenience static functions are provided for the creation of common lattice types from a minimum number of arguments. 

In [7]:
# Creates cubic Lattice with lattice parameter 4.2
lattice = mg.Lattice.cubic(4.2)
print(lattice.lengths_and_angles)

((4.2, 4.2, 4.2), (90.0, 90.0, 90.0))


A [Structure](http://pymatgen.org/pymatgen.core.structure.html?highlight=structure#pymatgen.core.structure.Structure) object represents a crystal structure (lattice + basis). A Structure is essentially a list of PeriodicSites with the same Lattice. Let us now create a CsCl structure.

In [8]:
structure = mg.Structure(lattice, ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
print("Unit cell vol = {}".format(structure.volume))
print("First site of the structure is {}".format(structure[0]))

Unit cell vol = 74.08800000000001
First site of the structure is [0. 0. 0.] Cs


The Structure object contains many useful manipulation functions. Since Structure is essentially a list, it contains a simple pythonic API for manipulation its sites. Some examples are given below. Please note that there is an immutable version of Structure known as IStructure, for the use case where you really need to enforce that the structure does not change. Conversion between these forms of Structure can be performed using from_sites().

In [9]:
del structure[0] # Remove the first site
structure.append("Na", [0,0,0]) # Append a Na atom.
structure[-1] = "Li" # Change the last added atom to Li.
structure[0] = "Cs", [0.01, 0.5, 0] # Shift the first atom by 0.01 in fractional coordinates in the x-direction.
immutable_structure = mg.IStructure.from_sites(structure) # Create an immutable structure (cannot be modified).
print(immutable_structure)

Full Formula (Cs1 Li1)
Reduced Formula: CsLi
abc   :   4.200000   4.200000   4.200000
angles:  90.000000  90.000000  90.000000
Sites (2)
  #  SP       a    b    c
---  ----  ----  ---  ---
  0  Cs    0.01  0.5    0
  1  Li    0     0      0


---
#### Exercise 2

In the console, print the `structure` variable, and make a copy of it called `supercell` using the `Structure.copy` method. Then check the docstring of the `make_supercell` method of the `supercell` variable using `Shift+Tab`. Use the method to create a 2x2x2 supercell and then print the `supercell` variable.

---

## Basic analyses

Pymatgen provides many analyses functions for `Structures`. Some common ones are given below.

In [10]:
# Determining the symmetry
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
finder = SpacegroupAnalyzer(structure)
print("The spacegroup is {}".format(finder.get_space_group_symbol()))

The spacegroup is Pmm2


It also has an extremely powerful structure matching tool.

In [11]:
from pymatgen.analysis.structure_matcher import StructureMatcher
# Let's create two structures which are the same topologically, but with different elements, and one lattice is larger.
s1 = mg.Structure(lattice, ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
s2 = mg.Structure(mg.Lattice.cubic(5), ["Rb", "F"], [[0, 0, 0], [0.5, 0.5, 0.5]])
m = StructureMatcher()
print(m.fit_anonymous(s1, s2)) # Returns a mapping which maps s1 and s2 onto each other. Strict element fitting is also available.

True


---
#### Exercise 3

The `SpacegroupAnalyzer` class has a lot of useful methods. Try initializing a `SpacegroupAnalyzer` object for the `supercell` variable, and then using the `get_conventional_standard_structure`. 

---

## Input/output

Pymatgen also provides IO support for various file formats in the pymatgen.io package. A convenient set of read_structure and write_structure functions are also provided which auto-detects several well-known formats. 

In [12]:
#Convenient IO to various formats. Format is intelligently determined from file name and extension.
structure.to(filename="CsCl.cif")

#Or if you just supply fmt, you simply get a string.
print(structure.to(fmt="cif"))

# generated using pymatgen
data_CsLi
_symmetry_space_group_name_H-M   'P 1'
_cell_length_a   4.20000000
_cell_length_b   4.20000000
_cell_length_c   4.20000000
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.00000000
_cell_angle_gamma   90.00000000
_symmetry_Int_Tables_number   1
_chemical_formula_structural   CsLi
_chemical_formula_sum   'Cs1 Li1'
_cell_volume   74.08800000
_cell_formula_units_Z   1
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
  Cs  Cs1  1  0.010000  0.500000  0.000000  1
  Li  Li2  1  0.000000  0.000000  0.000000  1



In [13]:
# Reading a structure from a file.
mg.Structure.from_file("CsCl.cif")

Structure Summary
Lattice
    abc : 4.2 4.2 4.2
 angles : 90.0 90.0 90.0
 volume : 74.08800000000001
      A : 4.2 0.0 2.571758278209442e-16
      B : -2.571758278209442e-16 4.2 2.571758278209442e-16
      C : 0.0 0.0 4.2
PeriodicSite: Cs (0.0420, 2.1000, 0.0000) [0.0100, 0.5000, 0.0000]
PeriodicSite: Li (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]

---
#### Exercise 4

As an extra to this tutorial, we're going to use the `imolecule` package to visualize the structure in a notebook. This requires that you first install the `imolecule` package. In a terminal, this can be done by:

```
pip install imolecule
```

The `imolecule` package relies on `openbabel` for translating the various structure formats. The most robust way to install `openbabel` is using a conda install:

```
conda install -c openbabel openbabel
```

Now import the `imolecule` package and visualize the CsCl.cif structure using the `imolecule.draw` method. Try writing the `supercell` structure to a different .cif file and visualize that.

---